# R-CNN minimal 
This notebook is a compact, well-commented re-creation of the `Object Detection using RCNN` notebook you provided. It is adapted to use the local `ex 9/data` layout and the filtered/pixel-annotated CSV created by the other notebook (`ex 9/data/interim/annotations_oi_filtered_pixels.csv`).

Notes for laymen: each code cell has clear comments. Run cells in order. If a cell fails because of missing packages (e.g., `selectivesearch`), follow the short install instructions in the next cell.

In [1]:
# If you need to install dependencies, uncomment and run the following line (one-time):
# !pip install selectivesearch torch torchvision pandas pillow matplotlib scikit-learn

# Lightweight imports used by the rest of the notebook
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import selectivesearch
import cv2
import torch
from torchvision import models, transforms
import random
import math
import matplotlib.pyplot as plt

# Paths (uses the data you have under ex 9/data)
ROOT = Path('.').resolve()
EX9 = ROOT / 'ex 9'
DATA = EX9 / 'data'
INTERIM = DATA / 'interim'
PIXELS_CSV = INTERIM / 'annotations_oi_filtered_pixels.csv'
IM_DIR = DATA / 'images'

print('Looking for pixel-annotated CSV at:', PIXELS_CSV)
print('Images dir:', IM_DIR)

Looking for pixel-annotated CSV at: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep learning lab\ex9\ex 9\data\interim\annotations_oi_filtered_pixels.csv
Images dir: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep learning lab\ex9\ex 9\data\images


In [2]:
# Load the pixel-annotated annotations that the other notebook creates.
# The CSV is expected to contain at least: ImageID, LabelName, XMinPx, XMaxPx, YMinPx, YMaxPx
if not PIXELS_CSV.exists():
    print('Pixel CSV not found at', PIXELS_CSV)
    print('Run the previous notebook to create:', PIXELS_CSV)
else:
    ann = pd.read_csv(PIXELS_CSV)
    display(ann.head())
Unique images in filtered CSV:', ann['ImageID'].nunique())

SyntaxError: unterminated string literal (detected at line 9) (448139515.py, line 9)

## Helper functions (IoU, selective search wrapper, image loader)
These helpers are small and explained for clarity. IoU measures overlap between two boxes. Selective search returns candidate boxes; we filter and convert them to the x1,y1,x2,y2 form.

In [ ]:
def iou(boxA, boxB, eps=1e-6):
    # boxes are (x1,y1,x2,y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    inter = interW * interH
    areaA = max(0, (boxA[2]-boxA[0])) * max(0, (boxA[3]-boxA[1]))
    areaB = max(0, (boxB[2]-boxB[0])) * max(0, (boxB[3]-boxB[1]))
    union = areaA + areaB - inter + eps
    return inter / union

def extract_selective_search_candidates(image, scale=200, min_size=100, max_candidates=500):
    # image: numpy RGB image
    img_lbl, regions = selectivesearch.selective_search(image, scale=scale, min_size=min_size)
    candidates = set()
    img_area = image.shape[0] * image.shape[1]
    for r in regions:
        x, y, w, h = r['rect']
        if w == 0 or h == 0:
            continue
        if r.get('size', 0) < 0.001 * img_area:
            continue
        # convert to x1,y1,x2,y2 and ensure in-bounds
        x1 = max(0, x)
        y1 = max(0, y)
        x2 = min(image.shape[1], x + w)
        y2 = min(image.shape[0], y + h)
        candidates.add((x1, y1, x2, y2))
        if len(candidates) >= max_candidates:
            break
    return list(candidates)

def load_image_for_id(image_id, images_dir=IM_DIR):
    # robust lookup: common extensions and recursive search
    p = Path(images_dir) / f'{image_id}.jpg'
    if not p.exists():
        p = Path(images_dir) / f'{image_id}.jpeg'
    if not p.exists():
        p = Path(images_dir) / f'{image_id}.png'
    if not p.exists():
        candidates = list(Path(images_dir).rglob(f'{image_id}.*'))
        p = candidates[0] if candidates else None
    if p is None or not p.exists():
        return None
    img = cv2.imread(str(p), cv2.IMREAD_COLOR)[..., ::-1]  # BGR->RGB
    return img

## Build a small R-CNN-style proposal dataset (safe demo)
We will: pick up to N images, generate proposals, match proposals to ground-truth boxes using IoU, label proposals (object class or background), and compute the simple bbox delta encoding used as regression targets.
This step can be slow if you run many images; it is safe to run with N=50 or N=100 for a CPU demo.

In [ ]:
def compute_delta(gt_box, prop_box, image_wh):
    # simple delta: (dx, dy, dw, dh) normalized by image size
    gx1, gy1, gx2, gy2 = gt_box
    px1, py1, px2, py2 = prop_box
    W, H = image_wh
    dx = (gx1 - px1) / max(1, W)
    dy = (gy1 - py1) / max(1, H)
    dw = (gx2 - px2) / max(1, W)
    dh = (gy2 - py2) / max(1, H)
    return [dx, dy, dw, dh]

def build_proposal_dataset(pixels_csv=PIXELS_CSV, N_images=80, iou_threshold_pos=0.5, max_props_per_image=300):
    if not Path(pixels_csv).exists():
        raise FileNotFoundError(f'Pixel CSV not found: {pixels_csv}')
    ann = pd.read_csv(pixels_csv)
    image_ids = ann['ImageID'].unique().tolist()
    random.shuffle(image_ids)
    image_ids = image_ids[:N_images]
    dataset = []
    for img_id in image_ids:
        img = load_image_for_id(img_id, IM_DIR)
        if img is None:
            print('Image not found for', img_id); continue
        H, W = img.shape[:2]
        df = ann[ann['ImageID'] == img_id]
        # ground-truth boxes for this image as x1,y1,x2,y2 (integers)
        gt_boxes = df[['XMinPx','YMinPx','XMaxPx','YMaxPx']].values.tolist()
        gt_labels = df['LabelName'].tolist()
        props = extract_selective_search_candidates(img, max_candidates=max_props_per_image)
        # for each prop find best IoU with any GT box
        for p in props:
            best_iou = 0.0
            best_idx = -1
            for gi, g in enumerate(gt_boxes):
                score = iou(p, g)
                if score > best_iou:
                    best_iou = score
                    best_idx = gi
            if best_iou >= iou_threshold_pos and best_idx >= 0:
                label = gt_labels[best_idx]  # MID; we keep MID for mapping later
                delta = compute_delta(gt_boxes[best_idx], p, (W,H))
            else:
                label = 'background'
                delta = [0,0,0,0]
            dataset.append({'image_id': img_id, 'prop': p, 'label': label, 'delta': delta, 'image_wh': (W,H)})
    print('Built proposal dataset with', len(dataset), 'proposals from', len(image_ids), 'images')
    return dataset

# Build a small dataset (change N_images to a smaller number if your CPU is slow)
small_ds = build_proposal_dataset(N_images=40, iou_threshold_pos=0.5, max_props_per_image=250)

# show a few examples to sanity check
for ex in small_ds[:4]:
    img = load_image_for_id(ex['image_id'], IM_DIR)
    x1,y1,x2,y2 = ex['prop']
    plt.figure(figsize=(6,4))
    plt.imshow(img)
    plt.gca().add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, edgecolor='r', facecolor='none', linewidth=2))
    plt.title(ex['label'])
    plt.axis('off')
    plt.show()

## Small R-CNN training skeleton (demo)
We use a pretrained VGG backbone for feature extraction on proposal crops, then train a small linear head for classification and a small regression head for bbox deltas. The training below is a *demo* and keeps things simple so it runs on CPU for small datasets.

In [ ]:
# Build label mapping (keep 'background' as index 0)
labels = sorted(set([d['label'] for d in small_ds]))
if 'background' not in labels:
    labels = ['background'] + labels
label2idx = {l:i for i,l in enumerate(labels)}
print('Labels:', labels)

# Simple dataset wrapper that returns crops, class idx and deltas
class RCNNProposalDataset(torch.utils.data.Dataset):
    def __init__(self, proposals, transform=None, image_dir=IM_DIR):
        self.proposals = proposals
        self.transform = transform or transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])
        self.image_dir = image_dir
    def __len__(self):
        return len(self.proposals)
    def __getitem__(self, idx):
        rec = self.proposals[idx]
        img = load_image_for_id(rec['image_id'], self.image_dir)
        x1,y1,x2,y2 = rec['prop']
        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            # fallback: return a zero tensor
            crop = np.zeros((224,224,3), dtype=np.uint8)
        # to PIL for torchvision transforms
        crop = Image.fromarray(crop)
        x = self.transform(crop)
        label = label2idx.get(rec['label'], 0)
        delta = torch.tensor(rec['delta'], dtype=torch.float32)
        return x, label, delta

# create dataset and dataloader (small batch size)
torch.manual_seed(0)
dataset = RCNNProposalDataset(small_ds)
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

# Build a tiny model: pretrained VGG features + heads
backbone = models.vgg16(pretrained=True)
# remove classifier and use features + adaptive pool to flatten
backbone.classifier = torch.nn.Identity()
for p in backbone.parameters():
    p.requires_grad = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
backbone = backbone.to(device).eval()

class SmallRCNN(torch.nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.backbone = backbone.features
        self.pool = torch.nn.AdaptiveAvgPool2d((7,7))
        self.flatten = torch.nn.Flatten()
        self.fc_cls = torch.nn.Linear(25088, n_classes)
        self.fc_reg = torch.nn.Linear(25088, 4)
    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flatten(x)
        cls = self.fc_cls(x)
        reg = self.fc_reg(x)
        return cls, reg

model = SmallRCNN(len(labels)).to(device)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
cls_loss_fn = torch.nn.CrossEntropyLoss()
reg_loss_fn = torch.nn.L1Loss()

# Demo training loop (single epoch, small dataset)
model.train()
for xb, yb, db in loader:
    xb = xb.to(device)
    yb = yb.to(device)
    db = db.to(device)
    opt.zero_grad()
    cls_pred, reg_pred = model(xb)
    loss_cls = cls_loss_fn(cls_pred, yb)
    # only compute reg loss on non-background examples
    mask = (yb != label2idx['background'])
    if mask.any():
        loss_reg = reg_loss_fn(reg_pred[mask], db[mask])
    else:
        loss_reg = torch.tensor(0., device=device)
    loss = loss_cls + 5.0 * loss_reg
    loss.backward()
    opt.step()
    print('Batch loss:', float(loss.detach().cpu().numpy()))
    break  # demo: run only one batch to show it works

## Next steps and notes
- The above cells give a working, small R-CNN style pipeline for *learning* purposes: data prep -> proposals -> labeling -> tiny training.
- For a full experiment: save `small_ds` to disk, implement full trainer with epochs/checkpoints, add evaluation (mAP) and faster RoI pooling for Fast R-CNN.

If you'd like, I can now: (A) wire the helper functions into a CLI script `scripts/prepare_openimages.py` to automate the data prep, (B) expand training to a full trainer with checkpointing and evaluation, or (C) add a small unit test for IoU and delta computation. Tell me which and I'll implement it next.